In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import matplotlib.pyplot as plt
from IPython.display import Image 
import os
import sys
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

DIR = "/content/drive/MyDrive/OMSCS/DL/FinalProject"
DATA_DIR = f'{DIR}/data'
NAME = 'mnist'
DATA_SET_CLASS = torchvision.datasets.MNIST
image_size = 28
in_channels = 1
fgsm_alpha = 0.5
fgsm_epsilon = 0.25

sys.path.append(os.path.abspath(DIR))

import team36
from team36.mnist.vgg import VGG
from team36.attacks.fast_gradient_attack_data_set import FastSignGradientAttackDataSet

In [8]:
model = VGG(image_size=image_size, in_channels=in_channels)
if torch.cuda.is_available():
    model = model.cuda()
state_dict = torch.load(f"{DIR}/checkpoints/{NAME}-vgg.pth")
model.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()

In [9]:
test_set = DATA_SET_CLASS(root=DATA_DIR, train=False, download=True, transform=transforms.ToTensor())
attack_test_set = FastSignGradientAttackDataSet(test_set, model, criterion, epsilon=fgsm_epsilon, device='cuda')

In [10]:
index = 1 # cifar10

input, ground_truth = test_set[index]
adversarial_input, _ = attack_test_set[index]

display(transforms.functional.to_pil_image(input))
display(transforms.functional.to_pil_image(adversarial_input))

image_dir = f"{DIR}/images"

pil_image = transforms.functional.to_pil_image(input)
pil_image.save(f"{image_dir}/{NAME}-{str(index)}.png")

pil_image = transforms.functional.to_pil_image(adversarial_input)
pil_image.save(f"{image_dir}/{NAME}-{str(index)}-{fgsm_epsilon}-fgsm-adversarial.png")